# Local Evaluation - Groundedness

After you have setup and configured the prompt flow, its time to evaluation its performance. Here we can use the prompt flow SDK to test different questions and see how the prompt flow performs using the evaluation prompt flows provided.

In [1]:
from promptflow import PFClient

pf_client = PFClient()

from dotenv import load_dotenv

load_dotenv()

True

In [2]:
# Add a question to test the base prompt flow.
question = "Can you tell me about your jackets?"
customerId = "4"
output = pf_client.test(
    flow="../contoso-chat", # Path to the flow directory
    inputs={ # Inputs to the flow
        "chat_history": [],
        "question": question,
        "customerId": customerId,
    },
)

output["answer"] = "".join(list(output["answer"]))

[2024-02-20 08:55:23,461][promptflow._sdk.entities._connection][WARNING] - Please use connection.secrets[key] to access secrets.


2024-02-20 08:55:23 +0000   17092 execution.flow     INFO     Start executing nodes in thread pool mode.
2024-02-20 08:55:23 +0000   17092 execution.flow     INFO     Start to run 5 nodes with concurrency level 16.
2024-02-20 08:55:23 +0000   17092 execution.flow     INFO     Executing node question_embedding. node run id: 339b88ff-c0f9-4ece-8689-71fa6a7634ab_question_embedding_0
2024-02-20 08:55:23 +0000   17092 execution.flow     INFO     Executing node customer_lookup. node run id: 339b88ff-c0f9-4ece-8689-71fa6a7634ab_customer_lookup_0
2024-02-20 08:55:24 +0000   17092 execution.flow     INFO     Node question_embedding completes.
2024-02-20 08:55:24 +0000   17092 execution.flow     INFO     Executing node retrieve_documentation. node run id: 339b88ff-c0f9-4ece-8689-71fa6a7634ab_retrieve_documentation_0
2024-02-20 08:55:25 +0000   17092 execution.flow     INFO     Node retrieve_documentation completes.
2024-02-20 08:55:25 +0000   17092 execution.flow     INFO     Node customer_looku

In [3]:
output

{'answer': "Of course, Sarah Lee! 😊 We have two amazing jackets that I think you'll love. \nFirst, we have the Summit Breeze Jacket 🏔️. It's a lightweight jacket perfect for outdoor adventures. It's windproof, water-resistant, and has adjustable cuffs for added comfort. Plus, its sleek black color adds style to function. 🌬️💦\n\nAnother great option is the RainGuard Hiking Jacket ☔. This jacket is made with waterproof, breathable fabric, so you'll stay dry and comfortable in any weather. It also has adjustable features and ventilation zippers for added convenience. And, it comes with reflective details for enhanced visibility during evening treks. 🌧️🌟\n\nBoth jackets are great choices for your outdoor activities! Let me know if you have any other questions or if there's anything else I can assist you with. 😊🌲",
 'context': [{'id': '3',
   'title': 'Summit Breeze Jacket',
   'content': "Discover the joy of hiking with MountainStyle's Summit Breeze Jacket. This lightweight jacket is your 

Test the groundedness of the prompt flow with the answer from the above question.

In [4]:
test = pf_client.test(
    flow="groundedness",
    inputs={
        "question": question,
        "context": str(output["context"]),
        "answer": output["answer"],
    },
)

2024-02-20 08:55:28 +0000   17092 execution.flow     INFO     Start executing nodes in thread pool mode.
2024-02-20 08:55:28 +0000   17092 execution.flow     INFO     Start to run 2 nodes with concurrency level 16.
2024-02-20 08:55:28 +0000   17092 execution.flow     INFO     Executing node groundedness_score. node run id: a581e286-231f-477c-9a45-cfe61651677a_groundedness_score_0
2024-02-20 08:55:30 +0000   17092 execution.flow     INFO     Node groundedness_score completes.
2024-02-20 08:55:30 +0000   17092 execution.flow     INFO     Executing node concat_scores. node run id: a581e286-231f-477c-9a45-cfe61651677a_concat_scores_0
2024-02-20 08:55:30 +0000   17092 execution.flow     INFO     Node concat_scores completes.
2024-02-20 08:55:30 +0000   17092 execution.flow     INFO     Start to run 1 nodes with concurrency level 16.
2024-02-20 08:55:30 +0000   17092 execution.flow     INFO     Executing node aggregate_variants_results. node run id: a581e286-231f-477c-9a45-cfe61651677a_aggre

In [5]:
test

{'gpt_groundedness': 5.0}

# Local Evaluation - Multiple Metrics 

Now use the same prompt flow and test it against the Multi Evaluation flow for groundedness, coherence, fluency, and relevance.

In [6]:
test_multi = pf_client.test(
    "multi_flow",
    inputs={
        "question": question,
        "context": str(output["context"]),
        "answer": output["answer"],
    },
)

2024-02-20 08:55:30 +0000   17092 execution.flow     INFO     Start executing nodes in thread pool mode.
2024-02-20 08:55:30 +0000   17092 execution.flow     INFO     Start to run 8 nodes with concurrency level 16.
2024-02-20 08:55:30 +0000   17092 execution.flow     INFO     Executing node coherence_score. node run id: b0917371-88aa-4caf-9fed-2e4778694de2_coherence_score_0
2024-02-20 08:55:30 +0000   17092 execution.flow     INFO     Executing node fluency_score. node run id: b0917371-88aa-4caf-9fed-2e4778694de2_fluency_score_0
2024-02-20 08:55:30 +0000   17092 execution.flow     INFO     Executing node groundedness_score. node run id: b0917371-88aa-4caf-9fed-2e4778694de2_groundedness_score_0
2024-02-20 08:55:30 +0000   17092 execution.flow     INFO     Executing node relevance_score. node run id: b0917371-88aa-4caf-9fed-2e4778694de2_relevance_score_0
2024-02-20 08:55:31 +0000   17092 execution.flow     INFO     Node coherence_score completes.
2024-02-20 08:55:31 +0000   17092 executi

In [7]:
test_multi

{'gpt_coherence': 5.0,
 'gpt_fluency': 5.0,
 'gpt_groundedness': 5.0,
 'gpt_relevance': 5.0}

# AI Studio Azure batch run on an evaluation json dataset

Now in order to test these more thoroughly, we can use the Azure AI Studio to run batches of test data with the evaluation prompt flow on a larger dataset.

In [8]:
import json
# Import required libraries
from promptflow.azure import PFClient

# Import required libraries
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

In [9]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

Populate the `config.json` file with the subscription_id, resource_group, and workspace_name.

In [10]:
config_path = "../config.json"
pf_azure_client = PFClient.from_config(credential=credential, path=config_path)

Found the config file in: ../config.json


Add the runtime from the AI Studio that will be used for the cloud batch runs.

In [11]:
# Update the runtime to the name of the runtime you created previously
runtime = "automatic"
# load flow
flow = "../contoso-chat"
# load data
data = "../data/salestestdata.jsonl"

In [12]:
# get current time stamp for run name
import datetime
now = datetime.datetime.now()
timestamp = now.strftime("%Y_%m_%d_%H%M%S")
run_name = timestamp+"chat_base_run"
print(run_name)

2024_02_20_085551chat_base_run


Create a base run to use as the variant for the evaluation runs. 

_NOTE: If you get "'An existing connection was forcibly closed by the remote host'" run the cell again._

In [13]:
# create base run in Azure Ai Studio
base_run = pf_azure_client.run(
    flow=flow,
    data=data,
    column_mapping={
        # reference data
        "customerId": "${data.customerId}",
        "question": "${data.question}",
    },
    runtime=runtime,
    # create a display name as current datetime
    display_name=run_name,
    name=run_name
)
print(base_run)

Uploading salestestdata.jsonl (< 1 MB): 100%|██████████| 465/465 [00:00<00:00, 2.96kB/s]


Uploading contoso-chat (0.68 MBs): 100%|██████████| 676856/676856 [00:01<00:00, 481511.34it/s]


[2024-02-20 08:55:56,435][promptflow][WARNING] - You're using automatic runtime, if it's first time you're using it, it may take a while to build runtime and you may see 'NotStarted' status for a while. 


Portal url: https://ai.azure.com/projectflows/bulkrun/run/2024_02_20_085551chat_base_run/details?wsid=/subscriptions/1e4ae802-8745-41ee-82bc-39495818793d/resourcegroups/contoso-chat-rg/providers/Microsoft.MachineLearningServices/workspaces/contoso-chat-sf-aiproj
name: 2024_02_20_085551chat_base_run
created_on: '2024-02-20T08:56:00.776171+00:00'
status: NotStarted
display_name: 2024_02_20_085551chat_base_run
description:
tags: {}
properties:
  azureml.promptflow.runtime_name: automatic
  azureml.promptflow.runtime_version: 20240205.v2
  azureml.promptflow.definition_file_name: flow.dag.yaml
  azureml.promptflow.flow_lineage_id: a6117409b8b717c7e79942d8573769a6d1f4ac180e7fabad302590d2b43bff3c
  azureml.promptflow.flow_definition_datastore_name: workspaceblobstore
  azureml.promptflow.flow_definition_blob_path: LocalUpload/9edf47f492fb06b9ce6ac78c9cfad6db/contoso-chat/flow.dag.yaml
  azureml.promptflow.inputs_mapping: '{"customerId":"${data.customerId}","question":"${data.question}"}'
  _

In [14]:
pf_azure_client.stream(base_run)

(Run status is 'NotStarted', continue streaming...)
(Run status is 'NotStarted', continue streaming...)
(Run status is 'NotStarted', continue streaming...)
(Run status is 'NotStarted', continue streaming...)
(Run status is 'NotStarted', continue streaming...)
(Run status is 'NotStarted', continue streaming...)
2024-02-20 08:59:40 +0000      51 promptflow-runtime INFO     [2024_02_20_085551chat_base_run] Receiving v2 bulk run request d8ac7d23-4bf1-489f-9973-551f3937c444: {"flow_id": "2024_02_20_085551chat_base_run", "flow_run_id": "2024_02_20_085551chat_base_run", "flow_source": {"flow_source_type": 1, "flow_source_info": {"snapshot_id": "0f0a6b6d-1c20-4089-bd16-5629229b16d5"}, "flow_dag_file": "flow.dag.yaml"}, "connections": "**data_scrubbed**", "log_path": "https://stcontoso38079365.blob.core.windows.net/5e451cdd-2273-4e66-91a3-e0129c84f7c8-azureml/ExperimentRun/dcid.2024_02_20_085551chat_base_run/logs/azureml/executionlogs.txt?sv=2019-07-07&sr=b&sig=**data_scrubbed**&skoid=8f51bbc3-

In [15]:
details = pf_azure_client.get_details(base_run)
details.head(10)

,inputs.customerId,inputs.question,inputs.line_number,inputs.chat_history,outputs.answer,outputs.context
outputs.line_number,,,,,,
0,4,tell me about your hiking jackets,0,[],Hey Sarah Lee! 🌲🧥\nOur hiking jackets are perf...,"[{'id': '17', 'title': 'RainGuard Hiking Jacke..."
1,1,Do you have any climbing gear?,1,[],"Hey John! 👋 Absolutely, we have some awesome c...","[{'id': '9', 'title': 'SummitClimber Backpack'..."
2,3,Can you tell me about your selection of tents?,2,[],Hey Michael! 👋 We've got a great selection of ...,"[{'id': '15', 'title': 'SkyView 2-Person Tent'..."
3,6,Do you have any hiking boots?,3,[],"Hey Emily! 👋 Absolutely, we have some fantasti...","[{'id': '4', 'title': 'TrekReady Hiking Boots'..."
4,2,What gear do you recommend for hiking?,4,[],"Hey Jane! 🌲 For hiking, I highly recommend the...","[{'id': '10', 'title': 'TrailBlaze Hiking Pant..."


# Cloud Eval run on Json Data

In [16]:
eval_flow = "multi_flow/"
data = "../data/salestestdata.jsonl"
run_name = timestamp+"chat_eval_run"
print(run_name)

eval_run_variant = pf_azure_client.run(
    flow=eval_flow,
    data=data,  # path to the data file
    run=base_run,  # use run as the variant
    column_mapping={
        # reference data
        "customerId": "${data.customerId}",
        "question": "${data.question}",
        "context": "${run.outputs.context}",
        # reference the run's output
        "answer": "${run.outputs.answer}",
    },
    runtime=runtime,
    display_name=run_name,
    name=run_name
)

2024_02_20_085551chat_eval_run


Uploading multi_flow (0.03 MBs): 100%|██████████| 31979/31979 [00:01<00:00, 25298.28it/s]


[2024-02-20 09:00:31,012][promptflow][WARNING] - You're using automatic runtime, if it's first time you're using it, it may take a while to build runtime and you may see 'NotStarted' status for a while. 


Portal url: https://ai.azure.com/projectflows/bulkrun/run/2024_02_20_085551chat_eval_run/details?wsid=/subscriptions/1e4ae802-8745-41ee-82bc-39495818793d/resourcegroups/contoso-chat-rg/providers/Microsoft.MachineLearningServices/workspaces/contoso-chat-sf-aiproj


In [17]:
pf_azure_client.stream(eval_run_variant)

(Run status is 'NotStarted', continue streaming...)
2024-02-20 09:01:04 +0000      51 promptflow-runtime INFO     [2024_02_20_085551chat_eval_run] Receiving v2 bulk run request 814d2872-3371-4680-8664-d49ae99829d2: {"flow_id": "2024_02_20_085551chat_eval_run", "flow_run_id": "2024_02_20_085551chat_eval_run", "flow_source": {"flow_source_type": 1, "flow_source_info": {"snapshot_id": "b935ddfa-1f6b-4474-9457-3b87f270da44"}, "flow_dag_file": "flow.dag.yaml"}, "connections": "**data_scrubbed**", "log_path": "https://stcontoso38079365.blob.core.windows.net/5e451cdd-2273-4e66-91a3-e0129c84f7c8-azureml/ExperimentRun/dcid.2024_02_20_085551chat_eval_run/logs/azureml/executionlogs.txt?sv=2019-07-07&sr=b&sig=**data_scrubbed**&skoid=8f51bbc3-8e23-4519-aa65-bed07b9f903a&sktid=d2f64b03-2cce-4f20-900e-b81248ebc1a8&skt=2024-02-20T08%3A45%3A59Z&ske=2024-02-21T16%3A55%3A59Z&sks=b&skv=2019-07-07&st=2024-02-20T08%3A51%3A03Z&se=2024-02-20T17%3A01%3A03Z&sp=rcw", "app_insights_instrumentation_key": "Instrume

In [18]:
details = pf_azure_client.get_details(eval_run_variant)
details.head(10)

,inputs.customerId,inputs.question,inputs.context,inputs.answer,inputs.line_number,inputs.chat_history,outputs.gpt_coherence,outputs.gpt_fluency,outputs.gpt_groundedness,outputs.gpt_relevance
outputs.line_number,,,,,,,,,,
0,4,tell me about your hiking jackets,"[{'id': '17', 'title': 'RainGuard Hiking Jacke...",Hey Sarah Lee! 🌲🧥\nOur hiking jackets are perf...,0,[],5.0,5.0,5.0,5.0
1,1,Do you have any climbing gear?,"[{'id': '9', 'title': 'SummitClimber Backpack'...","Hey John! 👋 Absolutely, we have some awesome c...",1,[],5.0,5.0,1.0,5.0
2,3,Can you tell me about your selection of tents?,"[{'id': '15', 'title': 'SkyView 2-Person Tent'...",Hey Michael! 👋 We've got a great selection of ...,2,[],5.0,5.0,5.0,5.0
3,6,Do you have any hiking boots?,"[{'id': '4', 'title': 'TrekReady Hiking Boots'...","Hey Emily! 👋 Absolutely, we have some fantasti...",3,[],5.0,5.0,5.0,5.0
4,2,What gear do you recommend for hiking?,"[{'id': '10', 'title': 'TrailBlaze Hiking Pant...","Hey Jane! 🌲 For hiking, I highly recommend the...",4,[],4.0,4.0,5.0,5.0


In [19]:

metrics = pf_azure_client.get_metrics(eval_run_variant)
print(json.dumps(metrics, indent=4))

{
    "gpt_coherence": 4.8,
    "gpt_coherence_pass_rate(%)": 100.0,
    "gpt_fluency": 4.8,
    "gpt_fluency_pass_rate(%)": 100.0,
    "gpt_groundedness": 4.2,
    "gpt_groundedness_pass_rate(%)": 80.0,
    "gpt_relevance": 5.0,
    "gpt_relevance_pass_rate(%)": 100.0
}


In [20]:
pf_azure_client.visualize([base_run, eval_run_variant])

Web View: https://ml.azure.com/prompts/flow/bulkrun/runs/outputs?wsid=/subscriptions/1e4ae802-8745-41ee-82bc-39495818793d/resourceGroups/contoso-chat-rg/providers/Microsoft.MachineLearningServices/workspaces/contoso-chat-sf-aiproj&runId=2024_02_20_085551chat_base_run,2024_02_20_085551chat_eval_run
